In [3]:
from datetime import date
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium import webdriver # driving the actions, links up with the browser
from bs4 import BeautifulSoup
import time
import re
from pick import pick
import progressbar
import os

In [10]:
# change the path

search_webpage = 'https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm'
PATH = r'C:\Users\esthe\Documents\Coding\chromedriver_win32\chromedriver.exe'
#driver = webdriver.Chrome(PATH)
#driver.get(search_webpage)

In [11]:
yes = ["Yes", "Y", "YES", "y", "yes"]
starting_page = "https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"
dashboard_url = "http://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"
login_url = 'https://cas.uwaterloo.ca/cas/login?service=https://waterlooworks.uwaterloo.ca/waterloo.htm'
output_name = 'WW_postings.csv'
choice = 'Application Deadlines in the next 10 Days'
potential_html1 = None
potential_html2 = None
potential_html3 = None

In [ ]:
{"Work Term Duration:": None, # might need to do some regex filtering
                                         "Job Summary:" : None,
                                         "Job Responsibilities:": None,
                                         "Required Skills:" : None,
                                         "Targeted Degrees and Disciplines:": None,
                                         "Application Documents Required:": None} 

In [17]:
df = pd.DataFrame(columns=['ID', 'Job Title', 'Organization', 'Division', 'Openings', 'Internal Status', 'City', 'Level',
                           'Applications', 'App Deadline', 'Work Term Duration', "Job Summary", "Job Responsibilities", "Required Skills", 
                           "Targeted Degrees and Disciplines", "Application Documents Required"])
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required


In [43]:
df.loc[3] = ["hello"] * 17

In [53]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required


In [38]:
def main():
    starting_page = "https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm"

    output_name = 'WW_postings.csv'
    
    browser = webdriver.Chrome(PATH)
    browser.get(starting_page) # in original script, this was starting page

    input('Are you ready?')

    choice = 'Viewed'
    
    data = get_job_lists(choice, browser, output_name)

    print("Done!")


def get_job_lists(choice, browser, output_name):
    
    print("Getting job lists ...")

    # Navigate to the job listings page then wait for the javascript to load
    browser.find_element_by_link_text(choice).click()

    # Get the HTML of the page and check if this is the last page (next_page_buttons)
    page_html = browser.execute_script("return document.body.innerHTML")
    pattern = r'<a href=".+?" onclick="loadPostingTable(.+?)">\s*»\s*<\/a>'
    next_page_buttons = []
    next_page_buttons = re.findall(pattern, page_html)



    # bar = progressbar.ProgressBar(max_value=self.job_lists_page_count)
    # bar.update(self.gather_current_progress)


    # dots for loading screen
    dots = ""

    # Scrape the tables and save it to output_name as a CSV
    job_posting_importants_order = ['Work Term Duration:', "Job Summary:", "Job Responsibilities:", "Required Skills:", 
                                    "Targeted Degrees and Disciplines:", "Application Documents Required:"]
    with open(output_name, 'w') as f:
        row_count = 0
        while (next_page_buttons):
            
            # loading screen stuff
            os.system('clear')
            print("Working" + dots)
            # update dots
            if (dots == "..."):
                dots = ""
            else:
                dots += "."

            # wait for JavaScript to load then download the page info
            time.sleep(2)
            soup = BeautifulSoup(page_html, "html.parser")
            for tr in soup.find_all('tr')[2:]: # iterating throw all rows
                
                # all the data that was not in the original script
                job_posting_importants = {"Work Term Duration:": None, # might need to do some regex filtering
                                         "Job Summary:" : None,
                                         "Job Responsibilities:": None,
                                         "Required Skills:" : None,
                                         "Targeted Degrees and Disciplines:": None,
                                         "Application Documents Required:": None} 
                
                
                tds = tr.find_all('td') # iterating all cells in each row.
                count = 0
                job_info_list = []
                for x in tds[0:-1]: # starting with the 3 buttons
                  
                    
                    if (count == 0): # at 3 buttons
                        link_container = x.find_all('a')[2]
              
                        onclick_link = link_container['onclick']
           
                        # clicking the onclick link:
                        browser.execute_script(onclick_link)
                        browser.switch_to.window(browser.window_handles[1])
                        
                        # give the browser 2 seconds for the website to catch up with the code
                        browser.implicitly_wait(2)
                        
                        time.sleep(3)
                        # do stuff, locate the texts of 'job posting importants'
                        
                        #saving the contents of the html of the new page:
                        indiv_html = browser.execute_script("return document.body.innerHTML")
                        indiv_posting = BeautifulSoup(indiv_html, "html.parser")
                        
                        # finding the third 'table' in the page, as the first table is no use to us
                        job_posting_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[1]
                        print("this is the job posting table")
                        #print(job_posting_information_table)
                       
                        
                        # iterating through all rows in the table
                        for row in job_posting_information_table.find_all('tr'): 
                            cell_list = row.find_all('td')
                            try:
                                key = cell_list[0].get_text().strip()
                            except:
                                continue
                        
        
                            if (key in job_posting_importants):
                                try:
                                    job_posting_importants[key] = cell_list[1].get_text().strip()
                                except:
                                    job_posting_importants[key] = ""
                    

                        application_information_table = indiv_posting.find_all('table', {'class': 'table-bordered'})[2]
                        job_posting_importants["Application Documents Required:"] = application_information_table.find_all('td')[3].get_text().strip()
    
                        browser.close() # close current tab
                        browser.switch_to.window(browser.window_handles[0])
                    
                        #print("lala")
                    if (count >= 2):  
                        #print(x.text)
                        job_info_list.append(str(x.text))
                        
                    count += 1
                        
                    
                for col in job_posting_importants_order:
                    job_info_list.append(job_posting_importants[col])
                
                df.loc[row_count] = job_info_list
                row_count+=1
                
            # navigate to the next page and get the HTML, next page buttons
            next_page_buttons = re.findall(pattern, page_html)
            browser.find_element_by_link_text("»").click()
            time.sleep(2)
            page_html = browser.execute_script("return document.body.innerHTML")


if __name__ == "__main__":
    main()

Are you ready?y
Getting job lists ...
Working
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting table
this is the job posting 

In [1]:
df

NameError: name 'df' is not defined

In [42]:
df.to_csv("ViewedJobs.xlsx")

ModuleNotFoundError: No module named 'openpyxl'

In [4]:
df = pd.read_csv("viewed.csv").drop(["Unnamed: 0"], axis=1)

In [5]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,185132,\n\n\n\t\t\t\t\t\t\t\t\t\tAccounting Analyst\n...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",27,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"<td width=""75%"">\n<span class=""np-view-questio..."
1,185136,\n\n\n\t\t\t\t\t\t\t\t\t\tSecurity\n\t\t\t\t\t...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",77,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment.\...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"<td width=""75%"">\n<span class=""np-view-questio..."
2,186638,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",64,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"<td width=""75%"">\n<span class=""np-view-questio..."
3,186639,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",188,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"<td width=""75%"">\n<span class=""np-view-questio..."
4,186640,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",41,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: \n\nConfidence: A...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"<td width=""75%"">\n<span class=""np-view-questio..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,185207,\n\n\n\t\t\t\t\t\t\t\t\t\tBackend Software Eng...,Oceanorth Technology Inc,Divisional Office,2,Open for Applications,Toronto,"Intermediate, Senior",4,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"As a backend Software Engineer at Oceanorth, y...",Develop robust operating systems that serve ou...,You have a degree in Computer Science or equiv...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"<td width=""75%"">\n<span class=""np-view-questio..."
196,186543,\n\n\n\t\t\t\t\t\t\t\t\t\tSoftware Engineering...,Okta Inc,Divisional Office,2,Open for Applications,Toronto,"Intermediate, Senior",47,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Role Location: Toronto \n\nPosition Descriptio...,Job Duties and Responsibilities:\n\nAnalyze/Re...,"Minimum Required Knowledge, Skills, and Abilit...",Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"<td width=""75%"">\n<span class=""np-view-questio..."
197,185640,\n\n\n\t\t\t\t\t\t\t\t\t\tData Scientist\n\t\t...,OMERS,Toronto Office,1,Expired - Apps Available,Toronto,"Junior, Intermediate, Senior",82,"\n\t\t\t\t\t\t\t\tJan 13, 2021\n\t\t\t\t\t\t\t...",4 month work term,Why join us?\nAre you looking to join an innov...,"As a member of this team, you will be responsi...","To succeed in this role, you have:

In [ ]:
'''
Things to Change
0. Get rid of whitespaces for job title - DONE
1. Get rid of capital NEW at the start of job titles - DONE
2. get rid of divison - DONE 
3. get rid of internal status - DONE
4. turn level into 3 binary columns - Dropped it, don't think its that important
5. filter all until only app deadline <= jan 19 - DONE
6. whitespace for targeted clusters - DONE
7. App docs required turns into need cover letter? - DONE
8. change Work Term Duration to [1,4]
9. whitepsace for app deadlinke - DONE

'''

In [64]:
def surrounding_space_remover(str):
    str = str.replace('\n','')
    str = str.replace('\t','')
    return str
    
df_space_remover = np.vectorize(surrounding_space_remover)

In [117]:

for column in ['Job Title', 'App Deadline', 'Targeted Degrees and Disciplines', "Job Summary", "Job Responsibilities", "Required Skills", "Targeted Degrees and Disciplines"]:
    df[column] = df_space_remover(df[column]) 
    

In [ ]:
df['Targeted Degrees and Disciplines']

In [74]:
def new_removal(str):
    try:
        if (str[0:3] == "NEW"): 
            return str[3:]
    except:
        return str # not enough chars in job title
    return str # if no NEW in job title

df_new_remover = np.vectorize(new_removal)

In [75]:
df['Job Title'] = df_new_remover(df["Job Title"])

In [77]:
df.drop(columns=['Division', 'Internal Status', 'Level'], inplace = True)

In [96]:
def day_extractor(str):
    str = str.replace(",", " ")
    str = [int(s) for s in str.split() if s.isdigit()]
    return str[0]

df_day_extractor = np.vectorize(day_extractor)

In [100]:
df['App Deadline'] = df_day_extractor(df["App Deadline"])
df = df[df["App Deadline"] <= 19]

AttributeError: 'numpy.int32' object has no attribute 'replace'

In [107]:
def cover_letter_needed(str):
    return ("Cover Letter" in str)

df_cover_letter_needed = np.vectorize(cover_letter_needed)

In [108]:
df['Application Documents Required'] = df_cover_letter_needed(df['Application Documents Required'])

In [110]:
df.rename(columns={"Application Documents Required": "Cover Letter Needed"}, inplace=True)

In [113]:
def work_term_duration(str):
    if (str == '4 month work term'): return 1
    if (str == '8 month consecutive work term required'): return 2
    if (str == '8 month consecutive work term preferred'): return 3
    else: return 4 # probably 4 month work term preferred?
    
df_work_term_duration = np.vectorize(work_term_duration)

In [114]:
df["Work Term Duration"] = df_work_term_duration(df["Work Term Duration"])

In [118]:
df

,ID,Job Title,Organization,Openings,City,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Cover Letter Needed
0,185132,Accounting Analyst,1Password,1,Toronto,27,19,1,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters- Theme - Accounting and Fina...,False
1,185136,Security,1Password,1,Toronto,77,19,1,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment.-...,Targeted Clusters- Theme - Computing: Software...,False
2,186638,QA Intern,1Password,1,Toronto,64,19,1,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table:- Confidence: A wi...,Targeted Clusters- Theme - Computing: Quality ...,False
3,186639,Junior Developer,1Password,1,Toronto,188,19,1,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table:- Confidence: A wi...,Targeted Clusters- Theme - Computing: Informat...,False
4,186640,1Password for Linux,1Password,1,Toronto,41,19,1,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: Confidence: A wil...,Targeted Clusters- Theme - Computing: Software...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,185207,Backend Software Engineering,Oceanorth Technology Inc,2,Toronto,4,19,1,"As a backend Software Engineer at Oceanorth, y...",Develop robust operating systems that serve ou...,You have a degree in Computer Science or equiv...,Targeted Clusters- Theme - Computing: Informat...,False
196,186543,Software Engineering Intern,Okta Inc,2,Toronto,47,19,1,Role Location: Toronto Position Description: W...,Job Duties and Responsibilities:Analyze/Refine...,"Minimum Required Knowledge, Skills, and Abilit...",Targeted Clusters- Theme - Computing: Software...,False
197,185640,Data Scientist,OMERS,1,Toronto,82,13,1,Why join us?Are you looking to join an innovat...,"As a member of this team, you will be responsi...","To succeed in this role, you have:Working towa...",Targeted Clusters- Theme - Computing: Informat...,True
198,187849,Data Scientist,Omnium LLC,2,Remote in Canada,56,19,1,Omnium is a boutique consulting and advisory f...,Omnium's internship program consists of the fo...,Must be studying in a STEM field.Successful in...,"Targeted Clusters- Theme - Data Analytics, Sta...",False


In [49]:
test = test.replace('\n','')
test = test.replace('\t','')
test

'Accounting Analyst'

In [46]:
test.replace(' ', '')

'\n\n\n\t\t\t\t\t\t\t\t\t\tAccountingAnalyst\n\t\t\t\t\t\t\t\t\t\n\n\n'

In [43]:
df

,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,185132,\n\n\n\t\t\t\t\t\t\t\t\t\tAccounting Analyst\n...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",27,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
1,185136,\n\n\n\t\t\t\t\t\t\t\t\t\tSecurity\n\t\t\t\t\t...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",79,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment.\...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
2,186638,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",64,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
3,186639,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",190,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
4,186640,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",43,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: \n\nConfidence: A...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,187337,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,Wattpad,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",20,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Wattpad is a global multiplatform entertainmen...,- Evolving Wattpad's data infrastructure\n- Im...,- Solid understanding of data engineering best...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
233,188451,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,William Ashley China Corp (A Fairfax Company),Divisional Office,1,Open for Applications,Vaughan,"Intermediate, Senior",9,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term required,William Ashley\nWilliam Ashley (WA) was founde...,"As noted above, WA has a pressing need for a n...",Excellent Excel proficiency; be able to write ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
234,187923,\n\n\t\t\t\t\t\t\t\t\t\tNEW\n\t\t\t\t\t\t\t\t\...,Wonolo Inc,Head Office,1,Open for Applications,Remote in Canada,"Intermediate, Senior",22,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Wonolo (Work Now Locally) is growing and disru...,- Iterate on existing CNN models and in-house ...,- Strong comprehe

In [53]:
ds = df.copy()



,ID,Job Title,Organization,Division,Openings,Internal Status,City,Level,Applications,App Deadline,Work Term Duration,Job Summary,Job Responsibilities,Required Skills,Targeted Degrees and Disciplines,Application Documents Required
0,185132,None,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",27,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...","You will assist with research, filing, and dat...",- 3rd or 4th-year co-op student majoring in Ac...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
1,185136,None,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",79,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"60,000 businesses and millions of people use 1...",- Developing or improving security-critical fe...,- Experience working in a remote environment.\...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
2,186638,None,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",64,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,- As developers finish their work on new featu...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
3,186639,None,1Password,Divisional Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",190,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,"We're looking for promising new developers, wh...",- Implement or improve features for the 1Passw...,What you bring to the table:\n- Confidence: A ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
4,186640,None,1Password,Divisional Office,1,Open for Applications,Toronto,"Intermediate, Senior",43,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,1Password is looking for promising new program...,Implement or improve features for the new 1Pas...,What you bring to the table: \n\nConfidence: A...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,187337,None,Wattpad,Head Office,1,Open for Applications,Toronto,"Junior, Intermediate, Senior",20,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Wattpad is a global multiplatform entertainmen...,- Evolving Wattpad's data infrastructure\n- Im...,- Solid understanding of data engineering best...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
233,188451,None,William Ashley China Corp (A Fairfax Company),Divisional Office,1,Open for Applications,Vaughan,"Intermediate, Senior",9,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",8 month consecutive work term required,William Ashley\nWilliam Ashley (WA) was founde...,"As noted above, WA has a pressing need for a n...",Excellent Excel proficiency; be able to write ...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
234,187923,None,Wonolo Inc,Head Office,1,Open for Applications,Remote in Canada,"Intermediate, Senior",22,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,Wonolo (Work Now Locally) is growing and disru...,- Iterate on existing CNN models and in-house ...,- Strong comprehension of statistical analysis...,Targeted Clusters\n\t\t\t\n\n\t\t\n\t\t\n\t\t\...,"University of Waterloo Co-op Work History,Résu..."
235,185105,None,XE.com,Divisional Office,1,Open for Applications,Newmarket,"Intermediate, Senior",15,"\n\t\t\t\t\t\t\t\tJan 19, 2021\n\t\t\t\t\t\t\t...",4 month work term,The Opportunity\n\nThis is a unique and exciti...,Job 

In [29]:
pip install openpyxl


Note: you may need to restart the kernel to use updated packages.


'c:\program' is not recognized as an internal or external command,
operable program or batch file.


In [45]:
a = {"bob": 2, 'sam':6, 'asd':123}
a.values()

dict_values([2, 6, 123])

In [31]:
' '.join("j, i, s".split())

'j, i, s'

In [ ]:
e2jin@uwaterloo.ca
Eevee1218.